In [8]:
import glob
import math
import os

import numpy as np
import pandas as pd

In [9]:
# Set Pandas display option to display all the columns
pd.set_option("display.max_columns", None)

In [10]:
# Generate the dataset and make minor adjustments

df = pd.read_csv("data/all_verbs_bhsa_bib_dss.csv").fillna("")
df["dir_he"] = df["dir_he"].astype(str)
df["dir_he"] = df["dir_he"].replace("1.0", "1").replace("0.0", "0")

df["dir_he_dss"] = df["dir_he_dss"].astype(str)
df["dir_he_dss"] = df["dir_he_dss"].replace("1.0", "1").replace("0.0", "0")
df.sort_values(["book", "chapter", "verse_num", "scroll"], ascending=[True, True, True, False], ignore_index=True, inplace=True)

In [11]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7'],
      dtype='object')

In [12]:
# Show the set of books present in the dataset
#set(df.book)
len(set(df.book))

60

In [14]:
df[df.book == "Leviticus"]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7
12607,52535,QRB[,MT,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,KJ JQRJB MKM QRBN L JHWH,,L JHWH,L JHWH/,0,prep,pers,anim,det,nmpr,simple,,,hif,impf,L,,,,,,
12608,52550,QRB[,MT,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,MN H BHMH MN H BQR W MN H Y>N TQRJBW >T QRBNKM,,MN H BHMH MN H BQR W MN H Y>N,MN H BHMH/ MN H BQR/ W MN H Y>N/,0,prep + prep + prep,,,det,subs,complex,,,hif,impf,MN,MN,MN,,,,
12609,1969253,QRB[,4Q25,Leviticus,1,2,JQRJB,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0,000 00 000 00000 0 0000 0000 000 00 00000 000 ...,hif,impf,,,,,,,
12610,1969268,QRB[,4Q25,Leviticus,1,2,TQRJBW,DBR >L BNJ JFR>L W >MRT >LHM >DM KJ JQRJB MKM ...,None,None,,,,,,,,,,0,000 00 000 00000 0 0000 0000 000 00 00000 000 ...,hif,impf,,,,,,,
12611,52561,QRB[,MT,Leviticus,1,3,JQRJBNW,>M <LH QRBNW MN H BQR ZKR TMJM JQRJBNW >L PTX ...,JQRJBNW,,no complement,no complement,,,,,,,,,,hif,impf,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13181,2103988,NPL[,Mas1b,Leviticus,9,24,JPLW,W TY> >C M L PNJ JHWH W T>KL <L H MZBX >T H <L...,W JPLW <L PNJHM,,<L PNJHM,,0,,,,,,,0,0 000 00 0 0 000 0000 0 0000 00 0 0000 00 0 00...,qal,wayq,<L,,,,,,
13182,57356,JY>[,MT,Leviticus,9,24,TY>,W TY> >C M L PNJ JHWH W T>KL <L H MZBX >T H <L...,W TY> >C M L PNJ JHWH,>C,M L PNJ JHWH,MN L PNH/ JHWH/,0,prep + prep,pers,anim,det,subs,complex,,,qal,wayq,MN,L,,,,,
13183,57382,NPL[,MT,Leviticus,9,24,JPLW,W TY> >C M L PNJ JHWH W T>KL <L H MZBX >T H <L...,W JPLW <L PNJHM,,<L PNJHM,<L PNH/,0,prep,,,det,subs,simple,,,qal,wayq,<L,,,,,,
13184,2081436,JY>[,11Q2,Leviticus,9,24,TY>,W TY> >C M L PNJ JHWH W T>KL <L H MZBX >T H <W...,None,None,,,,,,,,,,0,0 000 00 0 0 000 0000 0 0000 00 0 0000 00 0 00...,qal,wayq,,,,,,,


In [15]:
# Create one dataset per book (containing both BHSA and DSS rows)

unique_books = df['book'].unique()

# New columns to add

new_columns = [
    "cmpl_translation",  
    "cmpl_complex",
    "motion_type",
    "comments",
]

# Creating one dataset per unique book in the original dataset
for book in unique_books:
    book_df = df[df['book'] == book].copy()

    # Add each new column with NaN values
    for column in new_columns:
        book_df[column] = ''
        
    # Save the new dataset as a csv file
    book_df.to_csv(f"data/biblical_datasets/{book}_dataset.csv", index=False)

In [16]:
len(unique_books)

60